In [1]:
import tensorflow as tf

In [2]:
def my_network(input):
         W_1 = tf.Variable(tf.random_uniform([784, 100], -1, 1),
                          name="W_1")
         b_1 = tf.Variable(tf.zeros([100]), name="biases_1")
         output_1 = tf.matmul(input, W_1) + b_1
         W_2 = tf.Variable(tf.random_uniform([100, 50], -1, 1),
                          name="W_2")
         b_2 = tf.Variable(tf.zeros([50]), name="biases_2")
         output_2 = tf.matmul(output_1, W_2) + b_2
         W_3 = tf.Variable(tf.random_uniform([50, 10], -1, 1),
                          name="W_3")
         b_3 = tf.Variable(tf.zeros([10]), name="biases_3")
         output_3 = tf.matmul(output_2, W_3) + b_3
         # printing names
         print "Printing names of weight parameters"
         print W_1.name, W_2.name, W_3.name
         print "Printing names of bias parameters"
         print b_1.name, b_2.name, b_3.name
         return output_3


In [3]:
i_1 = tf.placeholder(tf.float32, [1000, 784], name="i_1")

In [4]:
my_network(i_1)

Printing names of weight parameters
W_1:0 W_2:0 W_3:0
Printing names of bias parameters
biases_1:0 biases_2:0 biases_3:0


<tf.Tensor 'add_2:0' shape=(1000, 10) dtype=float32>

In [5]:
i_2 = tf.placeholder(tf.float32, [1000, 784], name="i_2")

In [6]:
my_network(i_2)

Printing names of weight parameters
W_1_1:0 W_2_1:0 W_3_1:0
Printing names of bias parameters
biases_1_1:0 biases_2_1:0 biases_3_1:0


<tf.Tensor 'add_5:0' shape=(1000, 10) dtype=float32>

In [7]:
def layer(input, weight_shape, bias_shape):
     weight_init = tf.random_uniform_initializer(minval=-1, maxval=1)
     bias_init = tf.constant_initializer(value=0)
     W = tf.get_variable("W", weight_shape,
                         initializer=weight_init)
     b = tf.get_variable("b", bias_shape,
                         initializer=bias_init)
     return tf.matmul(input, W) + b
    
def my_network(input):
     with tf.variable_scope("layer_1"):
          output_1 = layer(input, [784, 100], [100])
     with tf.variable_scope("layer_2"):
          output_2 = layer(output_1, [100, 50], [50])
     with tf.variable_scope("layer_3"):
          output_3 = layer(output_2, [50, 10], [10])
     return output_3

In [8]:
i_1 = tf.placeholder(tf.float32, [1000, 784], name="i_1")

In [9]:
my_network(i_1)

<tf.Tensor 'layer_3/add:0' shape=(1000, 10) dtype=float32>

In [10]:
i_2 = tf.placeholder(tf.float32, [1000, 784], name="i_2")

In [11]:
 with tf.variable_scope("shared_variables") as scope:
         i_1 = tf.placeholder(tf.float32, [1000, 784], name="i_1")
         my_network(i_1)
         scope.reuse_variables()
         i_2 = tf.placeholder(tf.float32, [1000, 784], name="i_2")
         my_network(i_2)

In [12]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [25]:
with tf.device('/gpu:2'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0], shape=[2, 2], name='a')
  b = tf.constant([1.0, 2.0], shape=[2, 1], name='b')
  c = tf.matmul(a, b)
sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True))
sess.run(c)

array([[  5.],
       [ 11.]], dtype=float32)

In [51]:
def inference(x):
     init = tf.constant_initializer(value=0)
     W = tf.get_variable("W", [784, 10],
                         initializer=init)
     b = tf.get_variable("b", [10],
                         initializer=init)
     output = tf.nn.softmax(tf.matmul(x, W) + b)
     return output

In [13]:
def loss(output, y):
     dot_product = y * tf.log(output)
     # Reduction along axis 0 collapses each column into a single
     # value, whereas reduction along axis 1 collapses each row
     # into a single value. In general, reduction along axis i
     # collapses the ith dimension of a tensor to size 1.
     xentropy = -tf.reduce_sum(dot_product, reduction_indices=1)
     loss = tf.reduce_mean(xentropy)
     return loss

In [14]:
def evaluate(output, y):
    correct_prediction = tf.equal(tf.argmax(output, 1),
                                  tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [55]:
def training(cost, global_step):
    tf.scalar_summary("cost", cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [15]:
# Parameters
learning_rate = 0.01
training_epochs = 1000
batch_size = 100
display_step = 1

In [16]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [60]:
logPath = "/home/sweaterr/tensorflow-log/logistic2"

with tf.Graph().as_default():
        # mnist data image of shape 28*28=784
        x = tf.placeholder("float", [None, 784])
        # 0-9 digits recognition => 10 classes
        y = tf.placeholder("float", [None, 10])
        output = inference(x)
        cost = loss(output, y)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = training(cost, global_step)
        eval_op = evaluate(output, y)
        summary_op = tf.merge_all_summaries()
        saver = tf.train.Saver()
        sess = tf.Session()
        summary_writer = tf.train.SummaryWriter(logPath,
                                            graph_def=sess.graph_def)
        init_op = tf.initialize_all_variables()
        sess.run(init_op)
        
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                minibatch_x, minibatch_y = mnist.train.next_batch(batch_size)
                # Fit training using batch data
                feed_dict = {x : minibatch_x, y : minibatch_y}
                sess.run(train_op, feed_dict=feed_dict)
                # Compute average loss
                minibatch_cost = sess.run(cost, feed_dict=feed_dict)
                avg_cost += minibatch_cost/total_batch
            # Display logs per epoch step
            if epoch % display_step == 0:
                val_feed_dict = {
                     x : mnist.validation.images,
                     y : mnist.validation.labels
                }
                accuracy = sess.run(eval_op, feed_dict=val_feed_dict)
                print "Validation Error:", (1 - accuracy)
                summary_str = sess.run(summary_op, feed_dict=feed_dict)
                summary_writer.add_summary(summary_str,
                                           sess.run(global_step))
                saver.save(sess, logPath + "/model-checkpoint",
                           global_step=global_step)
        print "Optimization Finished!"
        test_feed_dict = {
             x : mnist.test.images,
             y : mnist.test.labels
        }
        accuracy = sess.run(eval_op, feed_dict=test_feed_dict)
        print "Test Accuracy:", accuracy


Validation Error: 0.15020018816
Validation Error: 0.129200220108
Validation Error: 0.120000183582
Validation Error: 0.113400280476
Validation Error: 0.110000252724
Validation Error: 0.105800271034
Validation Error: 0.104800224304
Validation Error: 0.102800250053
Validation Error: 0.101200222969
Validation Error: 0.099000275135
Validation Error: 0.0982002019882
Validation Error: 0.0968002080917
Validation Error: 0.0948002338409
Validation Error: 0.093600153923
Validation Error: 0.0916002392769
Validation Error: 0.0906002521515
Validation Error: 0.0896002054214
Validation Error: 0.0884002447128
Validation Error: 0.0882003307343
Validation Error: 0.0878002643585
Validation Error: 0.0872002243996
Validation Error: 0.0876002907753
Validation Error: 0.085400223732
Validation Error: 0.0848002433777
Validation Error: 0.0862002968788
Validation Error: 0.0870001912117
Validation Error: 0.0856001973152
Validation Error: 0.0846002101898
Validation Error: 0.0856002569199
Validation Error: 0.0838001

In [41]:
def layer(input, weight_shape, bias_shape):
        weight_stddev = (2.0/weight_shape[0])**0.5
#         init = tf.constant_initializer(value=0)
        w_init = tf.random_normal_initializer(stddev=weight_stddev)
        bias_init = tf.constant_initializer(value=0)
        W = tf.get_variable("W", weight_shape,
                            initializer=w_init)
        b = tf.get_variable("b", bias_shape,
                            initializer=bias_init)
        return tf.nn.relu(tf.matmul(input, W) + b)
    
def inference1(x):
    with tf.variable_scope("hidden_1"):
        hidden_1 = layer(x, [784, 256], [256])
    with tf.variable_scope("hidden_2"):
        hidden_2 = layer(hidden_1, [256, 256], [256])
    with tf.variable_scope("output"):
        output = layer(hidden_2, [256, 10], [10])
    return output

def inference(x):
    with tf.variable_scope("hidden_1"):
        hidden_1 = layer(x, [784, 10], [10])
    with tf.variable_scope("output"):
        output = layer(hidden_1, [10, 10], [10])
    return output

In [42]:
def loss(output, y):
     dot_product = y * tf.log(output)
     # Reduction along axis 0 collapses each column into a single
     # value, whereas reduction along axis 1 collapses each row
     # into a single value. In general, reduction along axis i
     # collapses the ith dimension of a tensor to size 1.
     xentropy = -tf.reduce_sum(dot_product, reduction_indices=1)
     loss = tf.reduce_mean(xentropy)
     return loss

def evaluate(output, y):
    correct_prediction = tf.equal(tf.argmax(output, 1),
                                  tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

def training(cost, global_step):
    tf.scalar_summary("cost", cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [47]:
# Parameters
learning_rate = 0.001
training_epochs = 1000
batch_size = 100
display_step = 1

In [48]:
logPath = "/home/sweaterr/tensorflow-log/feedforward"
with tf.Graph().as_default():
        # mnist data image of shape 28*28=784
        x = tf.placeholder("float", [None, 784])
        # 0-9 digits recognition => 10 classes
        y = tf.placeholder("float", [None, 10])
        output = inference(x)
        cost = loss(output, y)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = training(cost, global_step)
        eval_op = evaluate(output, y)
        summary_op = tf.merge_all_summaries()
        saver = tf.train.Saver()
        sess = tf.Session()
        summary_writer = tf.train.SummaryWriter(logPath,
                                            graph_def=sess.graph_def)
        init_op = tf.initialize_all_variables()
        sess.run(init_op)
        
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                minibatch_x, minibatch_y = mnist.train.next_batch(batch_size)
                # Fit training using batch data
                feed_dict = {x : minibatch_x, y : minibatch_y}
                sess.run(train_op, feed_dict=feed_dict)
                # Compute average loss
                minibatch_cost = sess.run(cost, feed_dict=feed_dict)
                avg_cost += minibatch_cost/total_batch
            # Display logs per epoch step
            if epoch % display_step == 0:
                val_feed_dict = {
                     x : mnist.validation.images,
                     y : mnist.validation.labels
                }
                accuracy = sess.run(eval_op, feed_dict=val_feed_dict)
                print "Validation Error:", (1 - accuracy)
                summary_str = sess.run(summary_op, feed_dict=feed_dict)
                summary_writer.add_summary(summary_str,
                                           sess.run(global_step))
                saver.save(sess, logPath +"/model-checkpoint",
                           global_step=global_step)
        print "Optimization Finished!"
        test_feed_dict = {
             x : mnist.test.images,
             y : mnist.test.labels
        }
        accuracy = sess.run(eval_op, feed_dict=test_feed_dict)
        print "Test Accuracy:", accuracy

Validation Error: 0.796000018716
Validation Error: 0.746200025082
Validation Error: 0.718400001526
Validation Error: 0.719400018454
Validation Error: 0.726600021124
Validation Error: 0.724600017071
Validation Error: 0.718800008297
Validation Error: 0.71439999342
Validation Error: 0.711800009012
Validation Error: 0.706200003624
Validation Error: 0.70300000906
Validation Error: 0.70039999485
Validation Error: 0.70140004158
Validation Error: 0.698400020599
Validation Error: 0.69580000639
Validation Error: 0.693800032139
Validation Error: 0.693000018597
Validation Error: 0.693200021982
Validation Error: 0.69340005517
Validation Error: 0.693000018597
Validation Error: 0.694000005722
Validation Error: 0.69420003891
Validation Error: 0.694400012493
Validation Error: 0.694200009108
Validation Error: 0.694800019264
Validation Error: 0.692999988794
Validation Error: 0.692200005054
Validation Error: 0.692200005054
Validation Error: 0.691000014544
Validation Error: 0.690600007772
Validation Error: